In [77]:
import glob
import os
import pandas as pd

In [78]:
import otf2
from otf2.events import *

In [79]:
scorep_result_dir = './scorep-results'

# get all files in the directory
scorep_result_dir = './scorep-results'
scorep_result_dir = glob.glob(os.path.join(scorep_result_dir, '*'))
print(scorep_result_dir)

['./scorep-results/bt.C.25.mpi_io_full', './scorep-results/bt.B.9.mpi_io_full', './scorep-results/bt.A.4.scp', './scorep-results/bt.B.9.scp', './scorep-results/bt.B.25.scp', './scorep-results/bt.C.9.scp', './scorep-results/bt.B.1.mpi_io_full', './scorep-results/bt.A.1.mpi_io_full', './scorep-results/bt.A.9.mpi_io_full', './scorep-results/bt.B.16.mpi_io_full', './scorep-results/bt.B.25.mpi_io_full', './scorep-results/bt.C.4.mpi_io_full', './scorep-results/bt.A.16.scp', './scorep-results/bt.C.1.scp', './scorep-results/bt.C.25.scp', './scorep-results/bt.B.4.mpi_io_full', './scorep-results/bt.C.4.scp', './scorep-results/bt.C.16.scp', './scorep-results/bt.B.1.scp', './scorep-results/bt.A.4.mpi_io_full', './scorep-results/bt.A.1.scp', './scorep-results/bt.B.4.scp', './scorep-results/bt.A.16.mpi_io_full', './scorep-results/bt.C.16.mpi_io_full', './scorep-results/bt.B.16.scp', './scorep-results/bt.A.9.scp', './scorep-results/bt.A.25.scp', './scorep-results/bt.A.25.mpi_io_full', './scorep-resul

In [122]:
sample_otf2_file_path = "./scorep-results/bt.C.4.mpi_io_full" + '/traces.otf2'
print(sample_otf2_file_path)

./scorep-results/bt.C.4.mpi_io_full/traces.otf2


In [123]:
def read_trace(trace_name):
    with otf2.reader.open(trace_name) as trace:
        return trace

In [124]:
sample_trace = read_trace(sample_otf2_file_path)

In [125]:
dir(sample_trace.definitions)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__otf2_wrapper__set_clock_properties',
 '__otf2_wrapper_funcptr__set_clock_properties',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_calling_context_properties',
 '_calling_contexts',
 '_callpath_parameters',
 '_callpaths',
 '_callsites',
 '_cart_coordinates',
 '_cart_dimensions',
 '_cart_topologies',
 '_comms',
 '_get',
 '_groups',
 '_interrupt_generators',
 '_io_file_properties',
 '_io_files',
 '_io_handles',
 '_io_paradigms',
 '_io_pre_created_handle_states',
 '_location_group_properties',
 '_location_groups',
 '_location_properties',
 '_locations',
 '_metric_class_recorders',
 '_metric_members',
 '_metrics',
 '_paradigm_prope

In [127]:
list(sample_trace.definitions.locations)

[Location [0] 'Master thread',
 Location [1] 'Master thread',
 Location [2] 'Master thread',
 Location [3] 'Master thread']

In [128]:
import otf2

# Open the OTF2 trace file
with otf2.reader.open(sample_otf2_file_path) as trace:
    # Iterate through all locations and print their details
    for location in trace.definitions.locations:
        print(f"Name: {location}")

Name: Location [0]: 'name': 'Master thread', 'type': LocationType.CPU_THREAD, 'number_of_events': 25155, 'group': LocationGroup [0] 'MPI Rank 0'
Name: Location [1]: 'name': 'Master thread', 'type': LocationType.CPU_THREAD, 'number_of_events': 25152, 'group': LocationGroup [1] 'MPI Rank 1'
Name: Location [2]: 'name': 'Master thread', 'type': LocationType.CPU_THREAD, 'number_of_events': 25152, 'group': LocationGroup [2] 'MPI Rank 2'
Name: Location [3]: 'name': 'Master thread', 'type': LocationType.CPU_THREAD, 'number_of_events': 25152, 'group': LocationGroup [3] 'MPI Rank 3'


In [133]:
from collections import defaultdict
import otf2
from otf2.events import Enter, Leave

def calculate_accumulated_function_time(sample_otf2_file_path):
    # Track total time and call counts per function (region)
    function_times = defaultdict(float)
    function_counts = defaultdict(int)
    
    # Track active function calls per thread/process (location)
    # Structure: {location: [(region, enter_time)]}
    location_stacks = defaultdict(list)

    with otf2.reader.open(sample_otf2_file_path) as trace:
        clock_resolution = trace.timer_resolution

        for location, event in trace.events:
            if isinstance(event, Enter):
                # Push the region and its start time onto the location's stack
                location_stacks[location].append((event.region, event.time))
                # Increment the call count for this region
                function_counts[event.region] += 1
            elif isinstance(event, Leave):
                if not location_stacks[location]:
                    raise RuntimeError(f"Warning: Unmatched Leave event in location {location.name}")

                # Pop the last Enter event for this location
                region, enter_time = location_stacks[location].pop()
                if region != event.region:
                    raise RuntimeError(f"Warning: Mismatched Enter/Leave events in location {location.name}")

                # Calculate elapsed time and accumulate
                elapsed_ticks = event.time - enter_time
                elapsed_sec = elapsed_ticks / clock_resolution
                function_times[region] += elapsed_sec

    # Print results
    for region in function_times:
        total_time = function_times[region]
        call_count = function_counts[region]
        print(f"Function: {region.name}")
        print(f"  Total Calls: {call_count}")
        print(f"  Total Time: {total_time:.6f} seconds")
        print(f"  Average Time per Call: {total_time / call_count:.6f} seconds\n" if call_count > 0 else "\n")


In [134]:
calculate_accumulated_function_time(sample_otf2_file_path)

Function: MPI_Init
  Total Calls: 4
  Total Time: 3.571143 seconds
  Average Time per Call: 0.892786 seconds

Function: MPI_Comm_size
  Total Calls: 4
  Total Time: 0.000007 seconds
  Average Time per Call: 0.000002 seconds

Function: MPI_Comm_rank
  Total Calls: 4
  Total Time: 0.000002 seconds
  Average Time per Call: 0.000000 seconds

Function: MPI_Comm_dup
  Total Calls: 8
  Total Time: 0.002375 seconds
  Average Time per Call: 0.000297 seconds

Function: MPI_Bcast
  Total Calls: 32
  Total Time: 0.003023 seconds
  Average Time per Call: 0.000094 seconds

Function: MPI_File_delete
  Total Calls: 1
  Total Time: 0.035998 seconds
  Average Time per Call: 0.035998 seconds

Function: MPI_Barrier
  Total Calls: 12
  Total Time: 0.112276 seconds
  Average Time per Call: 0.009356 seconds

Function: MPI_File_open
  Total Calls: 8
  Total Time: 0.233338 seconds
  Average Time per Call: 0.029167 seconds

Function: MPI_File_set_view
  Total Calls: 8
  Total Time: 0.002926 seconds
  Average Ti